# 综合评价（多属性决策）

综合评价有两个非常需要注意的地方：
- 属性权重
- 计算得分的方式

不管使用哪一个综合评价方法，论文里必须要出现权重和得分两个东西

In [2]:
import pandas as pd
import numpy as np

df = pd.read_excel("data\\evaluation.xlsx").drop("序号", axis=1)
df.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10
0,1.5,7.1,280,424,4,430,90,112,459,453
1,1.5,7.1,280,424,4,431,92,112,459,453
2,1.5,8.7,280,424,4,431,92,112,459,453
3,1.5,8.7,280,424,4,431,92,112,459,448
4,1.5,8.7,280,424,4,431,90,112,455,453


## 数据预处理

In [3]:
# 这个标准化的过程会在消除量纲的基础上，让结果全为正数
def min_max_scaling(df):
    return (df - df.min()) / (df.max() - df.min())

def reverse_min_max(column):
    return (column.max() - column) / (column.max() - column.min())

# 数据标准化
df = min_max_scaling(df)

# 负向数据正向化：我假定数据集的第八列、第九列、第十列为负向特征
# 将其转换为正向，保证整个数据集统一
df[['X_8', 'X_9', 'X_10']] = reverse_min_max(df[['X_8', 'X_9', 'X_10']])

df.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10
0,0.583333,0.0,1.0,0.0,1.0,0.0,0.0,0.992509,0.0,0.000000
1,0.583333,0.0,1.0,0.0,1.0,1.0,1.0,0.992509,0.0,0.000000
2,0.583333,0.5,1.0,0.0,1.0,1.0,1.0,0.992509,0.0,0.000000
3,0.583333,0.5,1.0,0.0,1.0,1.0,1.0,0.992509,0.0,0.714286
4,0.583333,0.5,1.0,0.0,1.0,1.0,0.0,0.992509,0.8,0.000000


## 0. 综合评价原理

In [4]:
# 情况1：没有权重（所有特征相同的权重）
df1 = df.copy()
df1['综合得分'] = df1.sum(axis=1)
df1.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,综合得分
0,0.583333,0.0,1.0,0.0,1.0,0.0,0.0,0.992509,0.0,0.000000,3.575843
1,0.583333,0.0,1.0,0.0,1.0,1.0,1.0,0.992509,0.0,0.000000,5.575843
2,0.583333,0.5,1.0,0.0,1.0,1.0,1.0,0.992509,0.0,0.000000,6.075843
3,0.583333,0.5,1.0,0.0,1.0,1.0,1.0,0.992509,0.0,0.714286,6.790128
4,0.583333,0.5,1.0,0.0,1.0,1.0,0.0,0.992509,0.8,0.000000,5.875843


In [5]:
# 情况2：最简单的主观赋权法
df2 = df.copy()
weights = np.array([[0.2, 0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.3]])
df2['综合得分'] = df2.dot(weights.T)
df2.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,综合得分
0,0.583333,0.0,1.0,0.0,1.0,0.0,0.0,0.992509,0.0,0.000000,0.515918
1,0.583333,0.0,1.0,0.0,1.0,1.0,1.0,0.992509,0.0,0.000000,0.715918
2,0.583333,0.5,1.0,0.0,1.0,1.0,1.0,0.992509,0.0,0.000000,0.815918
3,0.583333,0.5,1.0,0.0,1.0,1.0,1.0,0.992509,0.0,0.714286,1.030203
4,0.583333,0.5,1.0,0.0,1.0,1.0,0.0,0.992509,0.8,0.000000,0.795918



## 1. 层次分析法（主观赋权法）

**特点：**
- 将复杂的决策问题分解成层次结构，包括目标层、准则层和方案层。
- 通过成对比较矩阵来确定各层次因素的相对重要性。
- 计算特征向量作为权重，并通过加权求和得到各方案的综合得分。

**区别：**
- 适用于层次结构明确、需要处理主观判断的决策问题。
- 强调决策者的判断一致性。

**使用场景：**
- 项目优先级排序
- 供应商选择
- 复杂系统的评估与分析


判断矩阵在层次分析法（AHP）中扮演着重要的角色，它是用来表达各个因素之间相对重要性的一种工具。判断矩阵的构建需要基于专家的主观判断或者实际数据分析，因此在构建判断矩阵时需要注意一些事项，以确保评价的准确性和可靠性。

以下是构建判断矩阵时需要注意的几个要点：

1. **一致性：** 假设你有n个选项或者标准需要进行比较，那么你会构造一个n×n的一致性矩阵。在这个矩阵中，对角线上的元素都是1（因为每个选项与自己比较的重要性相同），而矩阵的其他位置(i,j)上的值表示第i个选项相对于第j个选项的重要程度比值。相应地，矩阵的位置(j,i)上的值则是位置(i,j)上值的倒数，以保持矩阵的互反性质。

2. **完整性：** 判断矩阵应该是完整的，即每个因素相对于其他因素的重要性都要有明确的判断。如果某些因素之间的重要性无法判断，则需要进一步讨论或者采取其他方法来处理。

3. **对称性：** 判断矩阵应该是对称的，即如果因素A相对于因素B的重要性被判断为x，那么因素B相对于因素A的重要性也应该是1/x。如果判断矩阵不是对称的，可能会导致分析结果不准确。

4. **避免过度估计或低估：** 在给出判断时，应该尽量避免过度估计或低估因素之间的重要性。过度估计或低估都可能导致最终评价结果偏离实际情况。

5. **参考一致性指标：** 在构建判断矩阵后，通常需要进行一致性检验，以确保判断矩阵的一致性程度。可以使用一致性比率（Consistency Ratio，CR）来评估判断矩阵的一致性，与随机一致性指标进行比较，如果CR值接近0，则说明判断矩阵具有较好的一致性。

总的来说，构建判断矩阵需要充分考虑专家意见、数据分析结果以及相关理论知识，以确保判断矩阵具有一致性、完整性和可靠性。

In [6]:
import numpy as np

# 构建判断矩阵：判断矩阵是一个方阵，原数据集m * n，则判断矩阵为n维方阵 
# 一致性检验对判断矩阵要求极高，很难通过
judgment_matrix = np.array([
    [1, 1/2, 1/3, 1/4, 1/5, 1/6, 1/7, 1/8, 1/9, 1/10],
    [2, 1, 1/2, 1/3, 1/4, 1/5, 1/6, 1/7, 1/8, 1/9],
    [3, 2, 1, 1/2, 1/3, 1/4, 1/5, 1/6, 1/7, 1/8],
    [4, 3, 2, 1, 1/2, 1/3, 1/4, 1/5, 1/6, 1/7],
    [5, 4, 3, 2, 1, 1/2, 1/3, 1/4, 1/5, 1/6],
    [6, 5, 4, 3, 2, 1, 1/2, 1/3, 1/4, 1/5],
    [7, 6, 5, 4, 3, 2, 1, 1/2, 1/3, 1/4],
    [8, 7, 6, 5, 4, 3, 2, 1, 1/2, 1/3],
    [9, 8, 7, 6, 5, 4, 3, 2, 1, 1/2],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
])

def ahp(matrix):
    eigvals, eigvecs = np.linalg.eig(matrix)
    max_eigval = np.max(eigvals).real
    max_eigvec = eigvecs[:, np.argmax(eigvals)].real
    weights = max_eigvec / np.sum(max_eigvec)

    # 这个函数最后计算出了权重
    return weights, max_eigval

# 计算权重
weights, max_eigval = ahp(judgment_matrix)

# 评分
score = np.dot(df, weights)

score

array([0.24076995, 0.42559429, 0.43520562, 0.64641097, 0.49849769,
       0.53505116, 0.63361343, 0.83308393, 0.84769632, 0.6758545 ,
       0.6094582 , 0.66197595, 0.5942285 , 0.70574341, 0.69586645,
       0.61959186, 0.59749707, 0.57726584])


## 2. 熵值法（客观赋权法）

**特点：**
- 利用熵值衡量属性的信息量，确定各属性的权重。
- 客观确定权重，不依赖于决策者的主观判断。

**区别：**
- 适用于属性之间数据客观、独立的情况。
- 强调数据的客观性和信息量的利用。

**使用场景：**
- 数据分析与评价
- 经济指标分析
- 科技成果评价


In [7]:
def entropy_method(df):
    epsilon = 1e-9

    k = 1 / np.log(df.shape[0])
    p = df / df.sum(axis=0)
    entropy = -k * (p * np.log(p + epsilon)).sum(axis=0)
    redundacy = 1 - entropy

    weights = redundacy / redundacy.sum()

    return weights

weights = entropy_method(df)
score = np.dot(df, weights)

score

array([0.34707494, 0.54456579, 0.56930934, 0.63440893, 0.58866755,
       0.47620378, 0.57685435, 0.69184831, 0.72066623, 0.58987427,
       0.5933053 , 0.65599687, 0.5182269 , 0.60652808, 0.82452287,
       0.73846577, 0.71267699, 0.59001718])

## 3. 熵权法-TOPSIS法（客观赋权法）

**特点：**
- 通过计算各方案与理想解和负理想解的距离，确定各方案的综合排序。
- 强调各方案与理想解的接近程度。

**区别：**
- 适用于属性之间数据客观、独立的情况。
- 强调数据的客观性和信息量的利用。

**使用场景：**
- 项目优先级排序
- 供应商选择
- 复杂系统的评估与分析

In [8]:
import numpy as np

def E_j_fun(data):
    # 计算熵值
    epsilon = 1e-8  # 防止除以零
    data_sum = data.sum(axis=0) + epsilon  # 防止分母为零
    P_ij = (data + epsilon) / data_sum  # 防止分子为零
    E_j = -np.where(P_ij == 0, 0, (P_ij * np.log(P_ij)) / np.log(data.shape[0] + epsilon)).sum(axis=0)
    return E_j


def topsis(data):
    # topsis综合评价
    E_j = E_j_fun(data)
    G_j = 1 - E_j
    W_j = G_j / G_j.sum()
    
    # 数据标准化
    data_std = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))
    
    # 加权标准化矩阵
    Z_ij = data_std * W_j
    
    # 最优解和最劣解
    Imax_j = Z_ij.max(axis=0)
    Imin_j = Z_ij.min(axis=0)
    
    # 计算每个方案与最优解和最劣解的距离
    Dmax_i = np.sqrt(((Z_ij - Imax_j)**2).sum(axis=1))
    Dmin_i = np.sqrt(((Z_ij - Imin_j)**2).sum(axis=1))
    
    # 综合评价值
    C_i = Dmin_i / (Dmax_i + Dmin_i)
    
    return C_i, W_j

# 假设df是一个pandas DataFrame对象

ret_topsis, weights = topsis(df.values)  # 调用函数



In [9]:
ret_topsis

array([0.47220281, 0.54438326, 0.54937853, 0.57394205, 0.58989679,
       0.39831145, 0.44974042, 0.53452469, 0.53765637, 0.45484958,
       0.48162414, 0.51995345, 0.44269116, 0.48807595, 0.70542396,
       0.6615639 , 0.65686893, 0.57003437])

In [10]:
weights

array([0.02881791, 0.04948709, 0.03094652, 0.13884262, 0.23735301,
       0.0860571 , 0.11143375, 0.06243261, 0.16348996, 0.09113942])